In [1]:
from typing import Tuple
import os

import numpy as np
import numpy.typing

import pandas as pd
from matplotlib import pyplot as plt 


import nidaqmx
import nidaqmx.system
import nidaqmx.system._collections
import nidaqmx.system._collections.physical_channel_collection
from nidaqmx.constants import AcquisitionType, ProductCategory, LineGrouping
from nidaqmx.constants import LoggingMode, LoggingOperation, READ_ALL_AVAILABLE

import ImagingRig1P as irp

%matplotlib qt
%load_ext autoreload
%autoreload 2

## Set scan information

Imaging with 450 nM LED and performing opto stimulation with 550 nm LED. Chroma ZET 488/561 filter set


In [2]:
genotype = 'test'
date = '28_10_2024'
fly = 'fly1'



In [3]:
# set stimulation parameters
scan_info = {'scan_name': 'opto_pulse',
            'led450': np.nan, # power in CoolLED units
            'led550': np.nan, # power in CoolLED units
            'sample_rate': 10000, # Hz
            'duration': 10, # s
            }

opto_stim_params = {'inter_stim_interval': 5, # seconds, time between rising edges of opto stim
                    'init_delay': 3, # seconds
                    'duration': .1, # seconds
                   }
scan_info['opto_stim_params'] = opto_stim_params

In [4]:
# number of samples in recording
n_samples = int(scan_info['sample_rate']*scan_info['duration'])

# time array
time = np.arange(0,scan_info['duration'],1./scan_info['sample_rate'])

# digital outputs
do_data = {'camera_trigger': np.zeros([n_samples,]),
           'led450_trigger': np.zeros([n_samples,]), # imaging
           'led550_trigger': np.zeros([n_samples,]), # opto
           'pump_trigger': np.zeros([n_samples,]), 
           }

# trigger camera
do_data['camera_trigger'][1:5000]=1

# imaging led
do_data['led450_trigger'][:-1]=1



# stim every 5 seconds
start_times = np.arange(opto_stim_params['init_delay'], scan_info['duration'], opto_stim_params['inter_stim_interval'])
end_times = start_times+opto_stim_params['duration']
for s, e in zip(start_times,end_times):
    t_s, t_e = np.argmin(np.abs(time-s)), np.argmin(np.abs(time-e))
    do_data['led550_trigger'][t_s:t_e]=1


# set filename
os.makedirs(f"D:\\{genotype}\\{date}\\{fly}\\", exist_ok=True)
num = 0
out_dir = f"D:\\{genotype}\\{date}\\{fly}\\{scan_info['scan_name']}_{num}.tdms"
while os.path.exists(out_dir):
    num +=1 
    out_dir = f"D:\\{genotype}\\{date}\\{fly}\\{scan_info['scan_name']}_{num}.tdms"
print(out_dir)

D:\test\28_10_2024\fly1\opto_pulse_7.tdms


### Run Experiment

In [5]:
exp = irp.experiment_builder.Experiment(do_data, out_dir,
                                        sample_rate = scan_info['sample_rate'], duration = scan_info['duration'])
exp.run_tasks()

(4, 100000)


### Load TDMS file and convert to pandas dataframe

In [6]:
#convert to dataframe
df = exp.tdms_to_dataframe()


In [8]:
# save dataframe with scan info as pickle file
exp.save_dataframe(df, scan_info)

In [9]:
df.head()

fig, ax = plt.subplots()
ax.plot(df['camera_trigger'])
ax.plot(df['led450_trigger'])
ax.plot(df['led550_trigger'])
fig.show()